In [ ]:
from transformers import AutoTokenizer
from rich import print
from onnxruntime import InferenceSession

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = InferenceSession("./weight/model.onnx")

In [ ]:
input_ids = tokenizer.encode("Hello, how are you?", return_tensors="pt")
print(input_ids)
# output = model.run()
# print(output)
# print(tokenizer.decode(output))

In [ ]:
model.run(["logits"], {"input_ids": input_ids.cpu().numpy()})

In [2]:
import onnxruntime
import numpy as np
import torch  # Assuming your input_ids is initially a PyTorch tensor

# --- Your problematic input_ids that caused the error ---
# Example: input_ids = torch.tensor([[15496, 11, 703, 389, 345, 30]])
# Let's create a dummy one with shape (1, 6) for demonstration
input_ids_tensor = torch.randint(0, 50257, (1, 6), dtype=torch.long)  # Shape [1, 6]
print(f"Original input_ids (PyTorch tensor) shape: {input_ids_tensor.shape}")

input_ids_np = input_ids_tensor.cpu().numpy()
print(f"Input_ids (NumPy array) shape for ONNX: {input_ids_np.shape}")


# Load the NEWLY EXPORTED ONNX model
onnx_model_path = (
    "./weight/model.onnx"  # Ensure this is the path to the re-exported model
)
model = onnxruntime.InferenceSession(onnx_model_path)

# Prepare input feed (only input_ids)
input_feed = {"input_ids": input_ids_np}

# Run inference
# Note: output_names should be a list, e.g., ["logits"]
try:
    outputs = model.run(["logits"], input_feed)
    print("ONNX Inference successful!")
    # The output logits shape should be [batch_size, sequence_length, vocab_size]
    # So for input shape [1, 6], output might be [1, 6, 50257]
    print(f"Logits output shape: {outputs[0].shape}")
except Exception as e:
    print(f"ONNX Inference failed: {e}")
    print("Ensure the model was re-exported with dynamic_axes for 'input_ids'.")
    print(
        "Verify ONNX model inputs using a tool like Netron or the verification code in the export script."
    )

Original input_ids (PyTorch tensor) shape: torch.Size([1, 6])
Input_ids (NumPy array) shape for ONNX: (1, 6)
ONNX Inference failed: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Got invalid dimensions for input: input_ids for the following indices
 index: 1 Got: 6 Expected: 1024
 Please fix either the inputs/outputs or the model.
Ensure the model was re-exported with dynamic_axes for 'input_ids'.
Verify ONNX model inputs using a tool like Netron or the verification code in the export script.
